# Merge Files
This script works with data in `sciebo\DTS Data\Alsdorf\Daten\DTS_unprocessed\Daten_Mario_oder_alt`, Alsdorf data channels 1 to 4.

This script converts the data exported with charon4 (export was not done by me), into a better format for processing them with python.
This script merges multiple smaller files into a bigger one.

the final filename: `temp_chx_year_month_vx.csv`
* temp: Temperature
* chx: Channel, x: number
* year
* month
* vx: version of this script

The main folder, which contains all files (about 500000) is `.zip`. It also contains the outputs of this script.
The data is still in the Charon4 Software and can be exported. It is also in `my_database`.
Therefore the data (the `.zip` file) is not needed, see also `compare.ipynb`.

## Outdated
this script is not needed anymore because the data in `Daten_Mario_oder_alt` is the same as in the used database.
Its just kept here if someone wants to check this again.

In [5]:
import numpy as np
import pandas as pd
import glob

from my_func_mvw.functions import get_abspath, file_len, number_files

# check of date deactivated because it takes very very long

In [10]:
activate_inputs=['temp_ch1_2018_6', 'temp_ch1_2018_7', 'temp_ch1_2018_11', 'temp_ch1_2019_2', 'temp_ch1_2019_3', 'temp_ch1_2019_4', 'temp_ch1_2019_5', 'temp_ch2_2018_6', 'temp_ch2_2018_7', 'temp_ch2_2019_2', 'temp_ch2_2019_3', 'temp_ch2_2019_4', 'temp_ch2_2019_5', 'temp_ch3_2018_6', 'temp_ch3_2019_2', 'temp_ch3_2019_3', 'temp_ch3_2019_4', 'temp_ch3_2019_5', 'temp_ch4_2018_6', 'temp_ch4_2019_2', 'temp_ch4_2019_3', 'temp_ch4_2019_4', 'temp_ch4_2019_5', 'temp_ch5_2018_6', 'temp_ch6_2018_6', 'temp_ch7_2018_11']

############################## Input############################################
# one or more of activate_inputs
# using all takes very long and may not use the most up-to-date version of this notebook
activate=activate_inputs[:]
#Example: activate_inputs[:]; activate_inputs[5]; activate_inputs[22:25]
path_wdir=r"..\Alsdorf\Daten\DTS_unprocessed\Daten_Mario_oder_alt" #path to working directory
#################################################################################

In [11]:
if type(activate) == str: # it is later assumed activate is a list of names
    activate=[activate]

# path which will be activated
# all files in each of these paths will be merged
paths_for_activate={}
# channel 1
paths_for_activate["temp_ch1_2018_6"]=path_wdir + r"\desktop-folder\Controller\3188\1\Temperature Data\2018\6"
paths_for_activate["temp_ch1_2018_7"]=path_wdir + r"\desktop-folder\Controller\3188\1\Temperature Data\2018\7"
paths_for_activate["temp_ch1_2018_11"]=path_wdir + r"\desktop-folder\Controller\3188\1\Temperature Data\2018\11"
paths_for_activate["temp_ch1_2019_2"]=path_wdir + r"\desktop-folder\Controller\3188\1\Temperature Data\2019\2"
paths_for_activate["temp_ch1_2019_3"]=path_wdir + r"\desktop-folder\Controller\3188\1\Temperature Data\2019\3"
paths_for_activate["temp_ch1_2019_4"]=path_wdir + r"\desktop-folder\Controller\3188\1\Temperature Data\2019\4"
paths_for_activate["temp_ch1_2019_5"]=path_wdir + r"\desktop-folder\Controller\3188\1\Temperature Data\2019\5"
# channel 2
paths_for_activate["temp_ch2_2018_6"]=path_wdir + r"\desktop-folder\Controller\3188\2\Temperature Data\2018\6"
paths_for_activate["temp_ch2_2018_7"]=path_wdir + r"\desktop-folder\Controller\3188\2\Temperature Data\2018\7"
paths_for_activate["temp_ch2_2019_2"]=path_wdir + r"\desktop-folder\Controller\3188\2\Temperature Data\2019\2"
paths_for_activate["temp_ch2_2019_3"]=path_wdir + r"\desktop-folder\Controller\3188\2\Temperature Data\2019\3"
paths_for_activate["temp_ch2_2019_4"]=path_wdir + r"\desktop-folder\Controller\3188\2\Temperature Data\2019\4"
paths_for_activate["temp_ch2_2019_5"]=path_wdir + r"\desktop-folder\Controller\3188\2\Temperature Data\2019\5"
# channel 3
paths_for_activate["temp_ch3_2018_6"]=path_wdir + r"\desktop-folder\Controller\3188\3\Temperature Data\2018\6"
paths_for_activate["temp_ch3_2019_2"]=path_wdir + r"\desktop-folder\Controller\3188\3\Temperature Data\2019\2"
paths_for_activate["temp_ch3_2019_3"]=path_wdir + r"\desktop-folder\Controller\3188\3\Temperature Data\2019\3"
paths_for_activate["temp_ch3_2019_4"]=path_wdir + r"\desktop-folder\Controller\3188\3\Temperature Data\2019\4"
paths_for_activate["temp_ch3_2019_5"]=path_wdir + r"\desktop-folder\Controller\3188\3\Temperature Data\2019\5"
# channel 4
paths_for_activate["temp_ch4_2018_6"]=path_wdir + r"\desktop-folder\Controller\3188\4\Temperature Data\2018\6"
paths_for_activate["temp_ch4_2019_2"]=path_wdir + r"\desktop-folder\Controller\3188\4\Temperature Data\2019\2"
paths_for_activate["temp_ch4_2019_3"]=path_wdir + r"\desktop-folder\Controller\3188\4\Temperature Data\2019\3"
paths_for_activate["temp_ch4_2019_4"]=path_wdir + r"\desktop-folder\Controller\3188\4\Temperature Data\2019\4"
paths_for_activate["temp_ch4_2019_5"]=path_wdir + r"\desktop-folder\Controller\3188\4\Temperature Data\2019\5"
# channel 5
paths_for_activate["temp_ch5_2018_6"]=path_wdir + r"\desktop-folder\Controller\3188\5\Temperature Data\2018\6"
# channel 6
paths_for_activate["temp_ch6_2018_6"]=path_wdir + r"\desktop-folder\Controller\3188\6\Temperature Data\2018\6"
# channel 7
paths_for_activate["temp_ch7_2018_11"]=path_wdir + r"\desktop-folder\Controller\3188\7\Temperature Data\2018\11"

In [12]:
def change_format(one_file,check_date=False):
    """
    changes the format of the imported data

    Only suited for the specific data used in this notebook
    """
    one_file = one_file.drop(columns=[one_file.columns[1], one_file.columns[2]])
    one_file = one_file.rename(columns={one_file.columns[0]: "Date"})
    #Turn the df as is expected to be
    one_file = pd.melt(one_file, id_vars="Date", var_name="Depth [m]", value_name="Temperature")
    #errors="ignore" #Creating a time series
    one_file["Date"] = pd.to_datetime(one_file["Date"], infer_datetime_format=True)
    #Converting an object to float
    one_file["Depth [m]"] = one_file["Depth [m]"].astype(float)
    # date is the same for all
    one_file = one_file.set_index(["Depth [m]"])

    # Very inefficient (computational) check if the file only contatains the same date
    # because the column will be dropped in the next step
    if check_date==True: #takes a lot of time
        for date in one_file["Date"]:
            for other_date in one_file["Date"]:
                if date != other_date:
                    print("Error: the file contains data from different dates")

    # Only one date remains, if check_date=False it is not checked if the dates are the same!
    # I checked it for a few files an for these all the dates have been the same    
    one_file.rename(columns = {'Temperature' : one_file["Date"][0]}, inplace = True)
    one_file=one_file.drop(columns=["Date"])

    return one_file

# Calculation - Most important cell

In [13]:
%%time
# with all: Wall time: 3h 44s on my i3 laptop

#Input
more_output=False # True False

#Calculation
counter=0
for name in activate:
    print("-----------------------------------------------------------------------")
    my_path=paths_for_activate[name]
    all_my_paths=get_abspath(my_path + "\*\*") # contains all paths to every file in the activated repository
    print("Start processing %.20s:"%name)
    # In some (a lot) files are at the end (multiple) duplicates of the data. 
    # The data is checked if these are realy duplicates. 

    all_files_len={} # number of lines in every file in the activated repository
    for file in all_my_paths:
        all_files_len[file] = file_len(file)

    #at the moment the notebook only works for 9, 18, 36!!! (or subset of these)
    for length in set(all_files_len.values()):
        if length not in {9, 18, 36}:
            print("ERROR: Script needs to be adapted for a new file length")
            print("length: %.5f"%length)

    n_files=number_files(my_path)
    print("number of files which will be compiled: %.1f"%n_files)

    # Read data, change format, compile and create new dataframe
    print("This may take some time")
    # start dataframe mustnt be empty, a little bit dirty but it works
    all_files = pd.read_csv(path_wdir + r"\desktop-folder\Controller\3188\1\Temperature Data\2018\6\25\2018-06-25-22-19-51.txt",skiprows=7)
    # Change Format
    all_files=change_format(all_files)
    all_files = all_files.rename(columns={all_files.columns[0]: "dupli"})

    if more_output==True:
        counter_process=0
        print("completion in %: not necessary proportional to computation time")
    for file in all_my_paths:
        if more_output==True:
            if counter_process % 20 == 0: # to get a feeling of remaining computation time
                print("%.1f " %(counter_process/n_files*100))
            counter_process+=1
        
        if all_files_len[file]==9: # no duplicates at the end
            # Read Data
            one_file = pd.read_csv(file,skiprows=7)
            # Change Format
            one_file=change_format(one_file)

        if all_files_len[file]==18: # one duplicates at the end
            # Read data
            # skipfooter does not work with default engine
            first = pd.read_csv(file, skiprows=7, skipfooter=9, engine="python")
            second = pd.read_csv(file, skiprows=16)
            # change the format
            first = change_format(first)
            second = change_format(second)
            # Check if all are equal
            # only checks if the shape and elements are the same. Index and columns names can be different
            # In this case this is sufficient I would say.
            if first.equals(second):
                one_file=first
            else:
                print("Error: 1")

        if all_files_len[file]==36: # three duplicates at the end
            # Read data
            first = pd.read_csv(file, skiprows=7, skipfooter=27, engine="python")
            second = pd.read_csv(file, skiprows=16,skipfooter=18, engine="python")
            third = pd.read_csv(file, skiprows=25, skipfooter=9, engine="python")
            fourth = pd.read_csv(file, skiprows=34)
            # change the format
            first = change_format(first)
            second = change_format(second)
            third = change_format(third)
            fourth = change_format(fourth)
            # Check if all are equal
            if first.equals(second) and first.equals(third) and first.equals(fourth):
                one_file=first
            else:
                print("Error: 2")

        all_files=pd.concat([all_files,one_file],axis=1)

    all_files=all_files.drop(columns="dupli") # a bit dirty but it works

    # Save compiled data
    filename=name + "_" + version + ".csv"
    counter+=1
    all_files.to_csv(path_wdir + "\\" + filename)  
    print("created %.40s"%filename)
    print("-----------------------------------------------------------------------")

-----------------------------------------------------------------------
Start processing temp_ch1_2018_6:
number of files which will be compiled: 691.0
This may take some time
created temp_ch1_2018_6_v1_2.csv
-----------------------------------------------------------------------
-----------------------------------------------------------------------
Start processing temp_ch1_2018_7:
number of files which will be compiled: 1944.0
This may take some time
created temp_ch1_2018_7_v1_2.csv
-----------------------------------------------------------------------
-----------------------------------------------------------------------
Start processing temp_ch1_2018_11:
number of files which will be compiled: 1238.0
This may take some time
created temp_ch1_2018_11_v1_2.csv
-----------------------------------------------------------------------
-----------------------------------------------------------------------
Start processing temp_ch1_2019_2:
number of files which will be compiled: 1486.0


# Load created data

In [14]:
# load new dataset
#df=pd.read_csv(filename,index_col=0)
#df